In [1]:
!pip install PuLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 31.7 MB/s eta 0:00:00


In [2]:
import networkx
import pulp
import numpy as np
import pandas as pd
import geopandas as gpd

from google.colab import drive
drive.mount('/content/drive')
path_opencp="drive/MyDrive/PDH | Thesis/Experiments/PredictCode/"
path_exp="drive/MyDrive/PDH | Thesis/Experiments/Fairness_Data/"

import sys, os.path, os
sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_exp))

os.chdir("drive/MyDrive/PDH | Thesis/Experiments/Fairness_Data/")

Mounted at /content/drive


#Loading Data

In [3]:
gdf_grilla = gpd.read_file('Bogota/Predictions/test_prediction_1.geojson',driver='GeoJSON')
localidades=gpd.read_file('Bogota/Raw Data/localidades_bogota.geojson')
geometrias=gpd.GeoSeries([i for i in localidades.geometry])
geo=geometrias.unary_union

gdf_grilla_filtrado = gdf_grilla[gdf_grilla.geometry.within(geo)].reset_index(drop=True)

gdf_grilla_filtrado.head()

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(
<ipython-input-3-a882b11f9c96>:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geo=geometrias.unary_union


,grid_id,tpg2017,densidad_urbana,Nombre de la localidad,pobreza_monetaria,uso,ESTRATO,uso_codificado,grid_id_matching_simetric,grid_id_matching_asimetric,lambda_predicha,geometry
0,18,60.07052,73.563395,BOSA,53.18,OTROS,1.375000,5780.0,19,227.0,1.616407e-14,"POLYGON ((-74.21051 4.61877, -74.21051 4.62773..."
1,58,60.07052,191.566568,BOSA,53.18,RESIDENCIAL,1.578431,8532.0,17,772.0,2.175105e-05,"POLYGON ((-74.20153 4.60982, -74.20153 4.61877..."
2,59,60.07052,266.317285,BOSA,53.18,RESIDENCIAL,1.489510,8532.0,100,812.0,1.269258e-05,"POLYGON ((-74.20153 4.61877, -74.20153 4.62773..."
3,60,60.07052,73.563395,BOSA,53.18,RESIDENCIAL,1.600000,8532.0,540,884.0,4.632100e-06,"POLYGON ((-74.20153 4.62773, -74.20153 4.63668..."
4,98,60.07052,309.569740,BOSA,53.18,RESIDENCIAL,1.776923,8532.0,139,723.0,2.599043e-05,"POLYGON ((-74.19254 4.60087, -74.19254 4.60982..."


In [4]:
gdf_grilla_filtrado.shape, gdf_grilla.shape

((567, 12), (984, 12))

In [5]:
def PMed_GDF(gdf_grilla, col_intensidad='lambda_predicha', id_col='grid_id', Ta=10, In=0.1, Th=5.0):
    # Extraer lista de IDs
    Ar = gdf_grilla[id_col].tolist()

    # Crear diccionario de llamadas (intensidad)
    Ca = gdf_grilla.set_index(id_col)[col_intensidad].to_dict()

    # Crear matriz de distancias entre centroides
    centroids = gdf_grilla.set_index(id_col).geometry.centroid
    Di = {
        i: {
            j: centroids[i].distance(centroids[j]) for j in Ar
        } for i in Ar
    }

    # Crear diccionario de contigüidad (vecinos)
    Co = {
        idx: list(gdf_grilla[gdf_grilla.touches(gdf_grilla.loc[i, 'geometry'])][id_col])
        for i, idx in enumerate(Ar)
    }

    # --- Resolver el problema ---
    SumCalls = sum(Ca.values())
    MaxIneq = (SumCalls / Ta) * (1 + In)
    MinIneq = (SumCalls / Ta) * (1 - In)
    print('Peso total:', SumCalls, 'Peso máx:', MaxIneq, 'Peso mín:', MinIneq)

    G = networkx.Graph()
    for i in Ar:
        for j in Co[i]:
            G.add_edge(i, j)

    NearAreas = {}
    Thresh = []
    for s in Ar:
        NearAreas[s] = []
        for d in Ar:
            if Di[s][d] < Th:
                Thresh.append((s, d))
                NearAreas[s].append(d)

    print('Thresh size:', np.shape(Thresh))

    P = pulp.LpProblem("P-Median", pulp.LpMinimize)
    assign_areas = pulp.LpVariable.dicts("Assign", Thresh, 0, 1, pulp.LpBinary)
    y_vars = pulp.LpVariable.dicts("Centers", Ar, 0, 1, pulp.LpBinary)

    P += pulp.lpSum(Ca[d] * Di[s][d] * assign_areas[(s, d)] for (s, d) in Thresh)
    P += pulp.lpSum(y_vars[s] for s in Ar) == Ta

    for (s, d) in Thresh:
        P += assign_areas[(s, d)] <= y_vars[s]
        if s != d:
            both = set(networkx.shortest_path(G, s, d)) & set(Co[d])
            nearer = [a for a in Co[d] if Di[s][a] < Di[s][d]]
            comb = list(both | set(nearer))
            P += pulp.lpSum(assign_areas[(s, a)] for a in comb if a in NearAreas[s]) >= assign_areas[(s, d)]

    for (sl, dl) in zip(Ar, Ar):
        P += pulp.lpSum(assign_areas[(s, dl)] for s in NearAreas[dl]) == 1
        P += pulp.lpSum(assign_areas[(sl, d)] * Ca[d] for d in NearAreas[sl]) <= MaxIneq
        P += pulp.lpSum(assign_areas[(sl, d)] * Ca[d] for d in NearAreas[sl]) >= MinIneq * y_vars[sl]

    print('Solving LP problem...')
    P.solve()
    stat = pulp.LpStatus[P.status]
    if stat != "Optimal":
        print("Status is %s" % (stat))
        return stat
    else:
        print("Status is %s, Total weighted travel: %d" % (stat, pulp.value(P.objective)))
        assignments = []
        for (s, d) in Thresh:
            if assign_areas[(s, d)].varValue == 1.0:
                assignments.append((s, d))

        df_result = pd.DataFrame(assignments, columns=['patrulla_id', 'grid_id'])
        gdf_result = gdf_grilla.merge(df_result, on='grid_id', how='left')
        return gdf_result


In [ ]:
# Parámetros del modelo
num_patrullas = 2         # Número de zonas o patrullas
inequidad_tolerada = 0.4   # 40% de tolerancia en carga de trabajo
umbral_distancia = 3000    # 3 km de umbral para considerar celdas cercanas

# Ejecutar el algoritmo de p-medianas
gdf_resultado = PMed_GDF(
    gdf_grilla=gdf_grilla_filtrado,
    col_intensidad='lambda_predicha',
    id_col='grid_id',
    Ta=num_patrullas,
    In=inequidad_tolerada,
    Th=umbral_distancia
)

<ipython-input-5-6d03de4df952>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gdf_grilla.set_index(id_col).geometry.centroid


Peso total: 0.011366893080906363 Peso máx: 0.007956825156634453 Peso mín: 0.0034100679242719087
Thresh size: (321489, 2)
Solving LP problem...


In [ ]:
gdf_resultado.plot(column='patrulla_id', cmap='jet', legend=True, figsize=(10,10))

In [ ]:
gdf_resultado.to_file('Bogota/Predictions/test_patrol_areas_1.geojson',driver='GeoJSON')